# L1: Hierarchical Content Generation

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.<br>
<span style="font-size: larger;">To maintain consistency, the notebooks are run with a 'world state' consistent with the video at the start of each notebook.</span></p>

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

## Azure OpenAI Setup

Before running this notebook, you need to set up your Azure OpenAI credentials. Create a `.env` file in the parent directory with:

```
AZURE_OPENAI_API_KEY=your_api_key_here
AZURE_OPENAI_ENDPOINT=https://your-resource-name.openai.azure.com/
AZURE_OPENAI_API_VERSION=2024-12-01-preview
```

**Important Setup Steps:**

1. **Get your credentials from Azure Portal:**
   - API Key: Found in your Azure OpenAI resource under "Keys and Endpoint"
   - Endpoint: Also found under "Keys and Endpoint" (format: https://your-resource-name.openai.azure.com/)

2. **Deploy a model:**
   - Go to Azure OpenAI Studio
   - Navigate to "Deployments" 
   - Deploy one of these models: `gpt-4o`, `gpt-4`, `gpt-35-turbo`, or `gpt-4o-mini`
   - Note the deployment name you choose

3. **Update model names:**
   - The code will automatically try common deployment names
   - If none work, manually update the model name in the code to match your deployment name

**Troubleshooting:**
- If you get a "404 Resource not found" error, your model deployment name doesn't match
- Check your deployment name in Azure OpenAI Studio under "Deployments"

## Creating a World

In [1]:
system_prompt = f"""
Your job is to help create interesting fantasy worlds that \
players would love to play in.
Instructions:
- Only generate in plain text without formatting.
- Use simple clear language without being flowery.
- You must stay below 3-5 sentences for each description.
"""

In [2]:
world_prompt = f"""
Generate a creative description for a unique fantasy world with an
interesting concept around cities build on the backs of massive beasts.

Output content in the form:
World Name: <WORLD NAME>
World Description: <WORLD DESCRIPTION>

World Name:"""


In [3]:
from openai import AzureOpenAI
from helper import get_azure_openai_config, load_env 


config = get_azure_openai_config()

# Add deployment name if missing
if "deployment" not in config:
    config["deployment"] = "o4-mini"

# Initialize client following Azure Portal pattern
client = AzureOpenAI(
    api_version=config["api_version"],
    azure_endpoint=config["azure_endpoint"],
    api_key=config["api_key"],
)

print("Configuration loaded:")
print(f"Endpoint: {config['azure_endpoint']}")
print(f"API Version: {config['api_version']}")
print(f"Deployment: {config['deployment']}")
print(f"API Key: {config['api_key'][:10]}...")

# Generate the world using the specific deployment
output = client.chat.completions.create(
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_prompt}
    ],
    max_completion_tokens=100000,
    model=config["deployment"]  # Use the deployment name from config
)

Configuration loaded:
Endpoint: https://Carson-4.openai.azure.com/
Endpoint: https://Carson-4.openai.azure.com/
API Version: 2024-12-01-preview
API Version: 2024-12-01-preview
Deployment: o4-mini
Deployment: o4-mini
API Key: d9c09c7888...
API Key: d9c09c7888...


In [4]:
world_output =output.choices[0].message.content
print(world_output)

World Name: Titan’s Trail  
World Description: On Titan’s Trail, colossal wandering beasts carry entire cities across endless dunes and brooding seas. Rival tribes tame and guide these leviathans, forging fragile alliances to chart safe passage. Citizens mine the creatures’ mineral-rich hides for building material and harvest their bioluminescent organs for light and power. When a beast stalls or packs clash, aerial guilds must broker peace or risk tumbling from the sky.


In [5]:
world_output = world_output.strip()
world = {
    "name": world_output.split('\n')[0].strip()
    .replace('World Name: ', ''),
    "description": '\n'.join(world_output.split('\n')[1:])
    .replace('World Description:', '').strip()
}

## Generating Kingdoms

In [6]:
kingdom_prompt = f"""
Create 3 different kingdoms for a fantasy world.
For each kingdom generate a description based on the world it's in. \
Describe important leaders, cultures, history of the kingdom.\

Output content in the form:
Kingdom 1 Name: <KINGDOM NAME>
Kingdom 1 Description: <KINGDOM DESCRIPTION>
Kingdom 2 Name: <KINGDOM NAME>
Kingdom 2 Description: <KINGDOM DESCRIPTION>
Kingdom 3 Name: <KINGDOM NAME>
Kingdom 3 Description: <KINGDOM DESCRIPTION>

World Name: {world['name']}
World Description: {world['description']}

Kingdom 1"""


In [12]:
output = client.chat.completions.create(
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": kingdom_prompt}
    ],
    max_completion_tokens=100000,
    model=config["deployment"]
)

In [14]:
kingdoms = {}
kingdoms_output = output.choices[0].message.content

for kingdom_block in kingdoms_output.split('\n\n'):
    kingdom_name = kingdom_block.strip().split('\n')[0] \
        .split('Name: ')[1].strip()
    print(f'Created kingdom "{kingdom_name}" in {world["name"]}')
    kingdom_description = kingdom_block.strip().split('\n')[1] \
        .split('Description: ')[1].strip()
    kingdom = {
        "name": kingdom_name,
        "description": kingdom_description,
        "world": world['name']
    }
    kingdoms[kingdom_name] = kingdom
world['kingdoms'] = kingdoms

# Print the description of the first kingdom
first_kingdom = list(kingdoms.values())[0]
print(f'\nKingdom 1 Description: \
{first_kingdom["description"]}')

Created kingdom "Sandspire Dominion" in Titan’s Trail
Created kingdom "Seastone Coalition" in Titan’s Trail
Created kingdom "Verdant Spine Principality" in Titan’s Trail

Kingdom 1 Description: The kingdom of Sandspire sits atop the Scarred Titan, a great desert leviathan that trembles through the Burning Dunes. Queen Sari Voss, a former sand-chaser, rules with a council of dune lords. Citizens build glass spires of fused hide and farm lumino outcrops along the creature’s spine. They are famed for taming storm horns to call sandstorms and broker peace with the Skywing Guild.


## Generating Towns

In [15]:
def get_town_prompt(world, kingdom):
    return f"""
    Create 3 different towns for a fantasy kingdom abd world. \
    Describe the region it's in, important places of the town, \
    and interesting history about it. \
    
    Output content in the form:
    Town 1 Name: <TOWN NAME>
    Town 1 Description: <TOWN DESCRIPTION>
    Town 2 Name: <TOWN NAME>
    Town 2 Description: <TOWN DESCRIPTION>
    Town 3 Name: <TOWN NAME>
    Town 3 Description: <TOWN DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description {kingdom['description']}
    
    Town 1 Name:"""


In [16]:
def create_towns(world, kingdom):
    print(f'\nCreating towns for kingdom: {kingdom["name"]}...')
    output = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_town_prompt(world, kingdom)}
        ],
        max_completion_tokens=100000,
        model=config["deployment"]
    )
    towns_output = output.choices[0].message.content
    
    towns = {}
    for output in towns_output.split('\n\n'):
        town_name = output.strip().split('\n')[0]\
        .split('Name: ')[1].strip()
        print(f'- {town_name} created')
        
        town_description = output.strip().split('\n')[1]\
        .split('Description: ')[1].strip()
        
        town = {
          "name": town_name,
          "description": town_description,
          "world": world['name'],
          "kingdom": kingdom['name']
        }
        towns[town_name] = town
    kingdom["towns"] = towns

In [17]:
for kingdom in kingdoms.values():
    create_towns(world, kingdom)  

town = list(kingdom['towns'].values())[0]
print(f'\nTown 1 Description: \
{town["description"]}')


Creating towns for kingdom: Sandspire Dominion...
- Mirageglass created
- Duneholm created
- Crestfall created
- Titan’s Trail created
- Sandspire Dominion created

Creating towns for kingdom: Seastone Coalition...
- Mirageglass created
- Duneholm created
- Crestfall created
- Titan’s Trail created
- Sandspire Dominion created

Creating towns for kingdom: Seastone Coalition...
- Brinehaven created
- Gillsmark created
- Saltvein created

Creating towns for kingdom: Verdant Spine Principality...
- Brinehaven created
- Gillsmark created
- Saltvein created

Creating towns for kingdom: Verdant Spine Principality...
- Mosswhisper Hollow created
- Emberfell Outpost created
- Luminbranch Crossing created

Town 1 Description: Mosswhisper Hollow clings to a mossy outcrop on the Forest Wanderer’s mid-canopy, where glowing vines drape like curtains. Its main sights are the Whispering Spire, a carved tree trunk where druids read the beast’s moods, and the Living Library, whose pages sprout new lea

## Generating Non-Player Characters (NPC's)

In [18]:
def get_npc_prompt(world, kingdom, town): 
    return f"""
    Create 3 different characters based on the world, kingdom \
    and town they're in. Describe the character's appearance and \
    profession, as well as their deeper pains and desires. \
    
    Output content in the form:
    Character 1 Name: <CHARACTER NAME>
    Character 1 Description: <CHARACTER DESCRIPTION>
    Character 2 Name: <CHARACTER NAME>
    Character 2 Description: <CHARACTER DESCRIPTION>
    Character 3 Name: <CHARACTER NAME>
    Character 3 Description: <CHARACTER DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description: {kingdom['description']}
    
    Town Name: {town['name']}
    Town Description: {town['description']}
    
    Character 1 Name:"""

In [26]:
def create_npcs(world, kingdom, town):
    print(f'\nCreating characters for the town of: {town["name"]}...')
    output = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_npc_prompt(world, kingdom, town)}
        ],
        max_completion_tokens=100000,
        model=config["deployment"],
        temperature=1
    )

    npcs_output = output.choices[0].message.content
    npcs = {}
    for output in npcs_output.split('\n\n'):
        npc_name = output.strip().split('\n')[0]\
        .split('Name: ')[1].strip()
        print(f'- "{npc_name}" created')
        
        npc_description = output.strip().split('\n')[1\
        ].split('Description: ')[1].strip()
        
        npc = {
        "name": npc_name,
        "description": npc_description,
        "world": world['name'],
        "kingdom": kingdom['name'],
        "town": town['name']
        }
        npcs[npc_name] = npc
    town["npcs"] = npcs

In [27]:
for kingdom in kingdoms.values():
    for town in kingdom['towns'].values():
        create_npcs(world, kingdom, town)
    # For now we'll only generate npcs for one kingdom
    break


Creating characters for the town of: Mirageglass...
- "Nara Varis" created
- "Jorin Marek" created
- "Ithos Callan" created

Creating characters for the town of: Duneholm...
- "Nara Varis" created
- "Jorin Marek" created
- "Ithos Callan" created

Creating characters for the town of: Duneholm...
- "Talan Vyrn" created
- "Varek Gildenhide" created
- "Jhara Stormcall" created

Creating characters for the town of: Crestfall...
- "Talan Vyrn" created
- "Varek Gildenhide" created
- "Jhara Stormcall" created

Creating characters for the town of: Crestfall...
- "Rasha Storm-Voice" created
- "Melun Vox" created
- "Dara Crescent-Lune" created

Creating characters for the town of: Titan’s Trail...
- "Rasha Storm-Voice" created
- "Melun Vox" created
- "Dara Crescent-Lune" created

Creating characters for the town of: Titan’s Trail...
- "Syra Nahl" created
- "Lord Karesh Delun" created
- "Mara Fen" created

Creating characters for the town of: Sandspire Dominion...
- "Syra Nahl" created
- "Lord Ka

In [28]:
npc = list(town['npcs'].values())[0]

print(f'\nNPC 1 in {town["name"]}, \
{kingdom["name"]}:\n{npc["description"]}')


NPC 1 in Sandspire Dominion, Sandspire Dominion:
Fenra is a lean, sun-tanned woman with braided silver hair and dust-cloud goggles perched on her brow. She works as an apprentice storm-horn tamer, coaxing the beasts that stir the dunes into fierce sandstorms. Haunted by the day her mentor’s horn shattered under her watch, she carries guilt he still blames on her. She yearns to master the storm horns and earn the queen’s approval to prove her worth.


## Save the World
>Note: You will save your world state to a file different than the one shown in the video to allow future lessons to be consistent with the video. If later wish to build your own worlds, you will want to load your file rather than the saved file.

In [33]:
import json
import os

def save_world(world, filename):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'w') as f:
        json.dump(world, f)

def load_world(filename):
    with open(filename, 'r') as f:
        return json.load(f)

#save_world(world, '../shared_data/Kyropeia.json')
save_world(world, 'shared_data/MatteoWorld_L1.json') #save to your version in subfolder